# Topic Modeling Text from 10ks

In [3]:
import re
import string
import os
import pickle
import mglearn
import spacy
spacy.prefer_gpu()

import pandas as pd
import numpy as np

import pyLDAvis.sklearn
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from tqdm.notebook import tqdm

/home/rlew/anaconda3/envs/rapids-0.17/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
# creating a list of the ciks we're interested in
f = open("final_ciks.txt", "r")
ciks = f.read().strip('[]').split(', ')
for cik in ciks:
    cik = str(cik).strip("'")
f.close()
ciks[:10]

["'1209028'",
 "'1069183'",
 "'825313'",
 "'1144980'",
 "'1800'",
 "'935036'",
 "'1113232'",
 "'796343'",
 "'7084'",
 "'1101215'"]

## Loading/Aggregating the Documents

In [13]:
try:
    hahahaha #don't run
    with open("combined_texts.p", "rb" ) as f:
        newtext = pickle.load(f)
except:
    merged_cik_texts = []
    for ind, cik in enumerate(tqdm(ciks)):
        # score_path = 'final_10ks/' + cik.strip("'") + '/rawtext/' + cik.strip("'") + '_sim_scores.csv'
        text_path = 'final_10ks/' + cik.strip("'") + '/merged_texts.p'
        with open(text_path, "rb" ) as f:
            cik_texts = pickle.load(f)
            merged_cik_texts += cik_texts
    print(f'The total number of documents is: {len(merged_cik_texts)}')
    newtext = []
    for entry in tqdm(merged_cik_texts):
        # getting rid of the terms seperately uses less memory
        entry = entry.replace('Form 10-K','')
        entry = entry.replace('FORM 10-K','')
        entry = entry.replace('10-K','')
        entry = re.sub(r'\d+', '', entry)#remove numbers
        entry = "".join([char.lower() for char in entry if char not in string.punctuation])
        #^^remove punctuation and make lowercase
        entry = "".join([char for char in entry if (not char.isdigit()) or (char == "'")])
        #^^remove numbers
        entry = entry.replace('table of contents','')
        entry = entry.replace('securities and exchange commission','')
        entry = entry.replace('annual report','')
        entry = entry.replace('loans','loan')
        entry = entry.replace('shares','share')
        entry = entry.replace('real estate','realestate')
        # entry = entry.replace('companys','company')
        #shouldn't be necessary if apostrophes are kept
        entry = entry.replace('million','')
        entry = re.sub('\s+', ' ', entry).strip() #remove doublespaces
        newtext.append(entry)
    with open("combined_texts.p", "wb" ) as f:
        pickle.dump(newtext, f)
    print(newtext[0][:400])

/home/rlew/anaconda3/envs/rapids-0.17/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:34: DeprecationWarning: invalid escape sequence \s
<>:34: DeprecationWarning: invalid escape sequence \s
<ipython-input-13-b8ebe306fab4>:34: DeprecationWarning: invalid escape sequence \s
  entry = re.sub('\s+', ' ', entry).strip() #remove doublespaces



The total number of documents is: 5619


## Lemmatizing, Removing Stop Words and Vectorizing

In [8]:
nlp = spacy.blank('en')
lemma_docs = []
doc_skip_count = 0
for document in tqdm(newtext):
    try:
        lemma_text = []
        lemma_doc = nlp(document)
        lemma_words = [token.lemma_.strip() for token in lemma_doc]
        for lemma_word in lemma_words:
            if (len(lemma_word) > 2):
                lemma_text.append(lemma_word)
        lemma_docs.append(' '.join(lemma_text))
    except:
        doc_skip_count += 1
        print(f'The document skip count is: {doc_skip_count}', end = '\r')
        
months = ['january','february','march','april','june','july','august','september','october','november','december']
freq_words = ['capital','management','creditors','brokerage','operating','product','products','business','value',
              'fair','statements','assets','operations','including','tax','consolidated','company', 'financial',
              'stock','cash','net','year', 'years','market','income','certain','related','costs','sales', 'new',
              'share', 'approximately', 'accounting', 'common', 'percent']
stop_words = text.ENGLISH_STOP_WORDS.union(months + freq_words)

vect=CountVectorizer(ngram_range=(1,1),stop_words=stop_words)
vectorized_corpus=vect.fit_transform(lemma_docs)
pd.DataFrame(final_corpus.toarray(),columns=vect.get_feature_names()).head()

The document skip count is: 5


## Performing LDA and Visualizing the Results

In [10]:
lda=LatentDirichletAllocation(n_components=6)
lda.fit_transform(final_corpus)

array([[9.45686205e-06, 1.69000528e-02, 9.45964372e-06, 9.45245309e-06,
        9.68861284e-04, 9.82102717e-01],
       [1.40698002e-04, 1.83590385e-02, 9.68342057e-05, 7.77942074e-06,
        1.58700399e-02, 9.65525610e-01],
       [4.64137344e-04, 7.82188703e-02, 2.85209801e-04, 4.91869040e-06,
        6.38122304e-02, 8.57214633e-01],
       ...,
       [4.80914179e-06, 4.80962003e-06, 4.81657837e-06, 1.47466312e-04,
        4.81110070e-06, 9.99833287e-01],
       [4.39284309e-06, 9.68558478e-04, 1.62975719e-04, 3.13684756e-04,
        1.86430962e-03, 9.96686079e-01],
       [7.34412944e-05, 7.34660116e-05, 9.62999513e-03, 7.34459154e-05,
        1.16959929e-01, 8.73189722e-01]])

In [11]:
sorting=np.argsort(lda.components_)[:,::-1]
features=np.array(vect.get_feature_names())
mglearn.tools.print_topics(topics=range(6), feature_names=features,sorting=sorting, topics_per_chunk=6, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       topic 5       
--------      --------      --------      --------      --------      --------      
development   properties    health        energy        fiscal        loan          
future        property      services      gas           ended         securities    
oil           ended         agreement     rate          plan          bank          
agreement     agreement     care          plan          services      credit        
research      partnership   ended         cost          results       losses        
ended         debt          notes         power         based         rate          
clinical      rate          plan          electric      revenue       total         
revenue       notes         based         total         credit        risk          
price         lease         information   ended         information   investment    
gas           filed         healthcare    credit        customers

In [12]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, vectorized_corpus, vect, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4       1.465019 -47.625065       1        1  34.480840
5      54.854797 -36.476795       2        1  21.091251
3      32.212929 -61.444572       3        1  14.472745
0       5.111096 -14.110332       4        1  12.852284
1      26.350170 -33.374702       5        1  10.091150
2      38.109818  -7.222781       6        1   7.011731, topic_info=              Term           Freq          Total Category  logprob  loglift
135370        loan  584916.000000  584916.000000  Default  30.0000  30.0000
77346       energy  121266.000000  121266.000000  Default  29.0000  29.0000
22148         bank  259787.000000  259787.000000  Default  28.0000  28.0000
97608          gas  139377.000000  139377.000000  Default  27.0000  27.0000
188815  properties  128911.000000  128911.000000  Default  26.0000  26.0000
...            ...            ...            ...      ...      ...      ...
123337        item   18572.905758  295332.134952   Topic6  -6.0355  -0.1088
82771      exhibit   17577.720986  217972.880428   Topic6  -6.0906   0.1398
56289       credit   18551.416197  397450.724917   Topic6  -6.0366  -0.4069
136520        loss   17908.656727  336370.237694   Topic6  -6.0719  -0.2753
204501      result   17703.587149  257146.663643   Topic6  -6.0834  -0.0183

[604 rows x 6 columns], token_table=        Topic      Freq       Term
term                              
295         1  0.000239     abbott
295         4  0.012257     abbott
295         6  0.987532     abbott
300         6  0.999808    abbotts
1040        2  0.005471        abx
...       ...       ...        ...
259608      6  0.023357  wisconsin
262037      6  0.999961       wynn
262160      3  0.999963       xcel
264657      1  0.999872        zar
265002      2  0.999610      zions

[1713 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 4, 1, 2, 3])